# Fine tuning


## Подготовка данных

In [ ]:
!pip install -q wandb

import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q --upgrade gdown

!gdown 1VPh3BJnMtAlymBJkE3n90AIG0Gk-UNC-
!unzip -q data_postprocessed.zip

Downloading...
From (original): https://drive.google.com/uc?id=1VPh3BJnMtAlymBJkE3n90AIG0Gk-UNC-
From (redirected): https://drive.google.com/uc?id=1VPh3BJnMtAlymBJkE3n90AIG0Gk-UNC-&confirm=t&uuid=d67ff186-102d-4bc1-b36d-e34ad06209c0
To: /content/data_postprocessed.zip
100% 44.0M/44.0M [00:02<00:00, 21.0MB/s]


In [ ]:
import json
import os
import random
from pathlib import Path
from pprint import pprint
from typing import List, Dict
from ast import literal_eval

import numpy as np

In [ ]:
np.random.seed(0)
random.seed(0)

In [ ]:
def list2str(l: List[str]) -> str:
    l = ['"' + i + '"' for i in l]
    l = ", ".join(l)
    return "[" + l + "]"


In [ ]:
entity2word = {
    "PER": "персона",
    "LOC": "место",
    "GEOPOLIT": "страна",
    "MEDIA": "медиа ресурс",
    "ORG": "организация",
    "LOCATION": "локация",
    "DATE": "дата",
    "AWARD": "награда",
    "FACILITY": "объект",
    "DISTRICT": "район",
    "EVENT": "событие",
    "WORK_OF_ART": "произведение искусства",
    "FAMILY": "семья",
    "LANGUAGE": "язык",
    "ORGANIZATION": "организация",
    "RELIGION": "религия",
    "PERSON": "персона",
    "DISEASE": "болезнь",
    "PERCENT": "процент",
    "COUNTRY": "страна",
    "IDEOLOGY": "идеология",
    "ORDINAL": "порядковый номер",
    "MONEY": "деньги",
    "PRODUCT": "продукт",
    "STATE_OR_PROVINCE": "штат или провинция",
    "PENALTY": "наказание",
    "NATIONALITY": "национальность",
    "AGE": "возраст",
    "LAW": "закон",
    "CRIME": "преступление",
    "PROFESSION": "профессия",
    "TIME": "время",
    "DI": "симптомы",
    "Drugform": "лекарственная форма",
    "Drugname": "название препарата",
    "Finding": "отсутствующий побочный эффект",
    "ADR": "побочный эффект",
    "Drugclass": "класс медикомета",
    'CHEM': "химическая субстанция",
    'LOCATION': "локация",
    'DATE': "дата",
    'PHYS': "физиологический процесс",
    'DEVICE': "медицинское устройство",
    'FINDING': "экспериментальное открытие",
    'HEALTH_CARE_ACTIVITY': "здравоохранение",
    'ADMINISTRATION_ROUTE': "пути введения лекарств",
    'GENE': "гены",
    'ACTIVITY': "поведение",
    'LABPROC': "диагностический метод",
    'MEDPROC': "терапевтическая процедура",
    'LIVB': "организм",
    'MENTALPROC': "ментальный процесс",
    'INJURY_POISONING': "повреждение",
    'NUMBER': "число",
    'DISO': "паталогический процесс",
    'SCIPROC': "исследовательский процесс",
    'NATIONALITY': "национальность",
    'AGE': "возраст",
    'LAW': "закон",
    'CRIME': "преступление",
    'TIME': "время",
    'ANATOMY': "структура тела",
    'PROFESSION': "профессия",
    "CITY": "город",
    "FOOD": "еда"
}

entity2word = {i.lower(): entity2word[i].lower() for i in entity2word}

In [ ]:
def apply_freq_based_neg_sampling(data, num2append):
    num_all_entities = 0
    num_negative = 0
    if "predicted_entities" in data[0]:
        entities_colums = "predicted_entities"
    else:
        entities_colums = "entities"


    entity_counter = {}
    for index_data, sample in enumerate(data):
        for index_pred, pred in enumerate(sample[entities_colums]):
            if pred[1].lower() not in entity_counter:
                entity_counter[pred[1].lower()] = 0
            entity_counter[pred[1].lower()] += 1
    types = []
    freq = []
    for t, f in entity_counter.items():
        types.append(t)
        freq.append(f)
    types = np.array(types)
    freq = np.array(freq)
    freq = freq / freq.sum()

    for index_data, sample in enumerate(data):
        num_all_entities += len(sample[entities_colums])
        num_negative += num2append

        unique_types_pred = list(set([i[1].lower() for i in sample[entities_colums]]))

        freq_corrected = np.copy(freq)
        bad_indexes = [np.where(types == i)[0][0] for i in unique_types_pred]
        freq_corrected[bad_indexes] = 0
        if freq_corrected.sum() == 0:
            continue
        freq_corrected = freq_corrected/freq_corrected.sum()

        sampled = np.random.choice(len(types), num2append, p=freq_corrected)
        sampled = np.unique(types[sampled])

        for new_type in sampled:
            data[index_data][entities_colums].append(["", new_type])
            random.shuffle(data[index_data][entities_colums])
    print("Общее кол-во предсказаний:", num_all_entities)
    print("Кол-во негативных:", num_negative)
    print("Доля негативных к общему числу:", num_negative/(num_negative+num_all_entities))
    print("--------------------------------------")
    return data


In [ ]:
spetial_tokens = ["<s>", "</s>", "<unk>", "<s>", "<unk>", "\n"]

def create_instructions(path: str, neg_sampling: bool, num_negative: int) -> List[Dict]:
    dataset = []
    dataset_name = path.split("/")[-2]
    split_json = json.load(open(path, "r"))
    if neg_sampling:
        split_json = apply_freq_based_neg_sampling(split_json, num2append=num_negative)
    for sample in split_json:
        try:
            conversations = []
            prompt_text = sample['prompt_text']
            for s in spetial_tokens:
                prompt_text = prompt_text.replace(s, "")
            conversations.append({
                "role": "user",
                "content": f"{prompt_text}"
            })
            conversations.append({
                "role": "bot",
                "content": "Я ознакомился с текстом."
            })
            try:
                lowered_sample = [(ner_text, ner_type.lower()) for ner_text, ner_type in sample["predicted_entities"]]
            except:
                lowered_sample = [(ner_text, ner_type.lower()) for ner_text, ner_type in sample["entities"]]

            for s in spetial_tokens:
                lowered_sample = [(ner_text.replace(s, ""), ner_type.replace(s, "")) for ner_text, ner_type in lowered_sample]

            if dataset_name != "gpt":
                lowered_sample = [(ner_text, entity2word[ner_type]) for ner_text, ner_type in lowered_sample]


            ner_type2entities = {}
            for ner_text, ner_type in lowered_sample:
                if ner_type not in ner_type2entities:
                    ner_type2entities[ner_type] = []
                if ner_text != "":
                    ner_type2entities[ner_type].append(ner_text)



            for ner_type, entities in ner_type2entities.items():
                conversations.append({
                    "role": "user",
                    "content": f"Что описывает {ner_type} в тексте?"
                })
                if len(entities) > 0:
                    conversations.append({
                        "role": "bot",
                        "content": list2str(entities)
                    })
                else:
                    conversations.append({
                        "role": "bot",
                        "content": "[]"
                    })
            dataset.append({
                "source": f"gpt",
                "messages": conversations
                })
        except Exception as e:
            print(str(e))
    return dataset

In [ ]:
gpt_train = create_instructions("data_postprocessed/gpt/train.json", neg_sampling=True, num_negative=2)
ne5_train = create_instructions("data_postprocessed/ne5/train.json", neg_sampling=True, num_negative=6)
nerel_train = create_instructions("data_postprocessed/nerel/train.json", neg_sampling=True, num_negative=6)
nerel_bio_train = create_instructions("data_postprocessed/nerel_bio/train.json", neg_sampling=True, num_negative=6)
person1000_train = create_instructions("data_postprocessed/person1000/train.json", neg_sampling=True, num_negative=6)
rudrec_train = create_instructions("data_postprocessed/rudrec/train.json", neg_sampling=True, num_negative=1)

gpt_val = create_instructions("data_postprocessed/gpt/val.json", neg_sampling=False, num_negative=0)

Общее кол-во предсказаний: 99412
Кол-во негативных: 30926
Доля негативных к общему числу: 0.23727539167395542
--------------------------------------
'entities'
'entities'
'entities'
'entities'
'entities'
Общее кол-во предсказаний: 22315
Кол-во негативных: 5100
Доля негативных к общему числу: 0.1860295458690498
--------------------------------------
Общее кол-во предсказаний: 50668
Кол-во негативных: 5034
Доля негативных к общему числу: 0.09037377472981221
--------------------------------------
Общее кол-во предсказаний: 60383
Кол-во негативных: 4134
Доля негативных к общему числу: 0.06407613497217787
--------------------------------------
Общее кол-во предсказаний: 22379
Кол-во негативных: 5100
Доля негативных к общему числу: 0.18559627351795918
--------------------------------------
Общее кол-во предсказаний: 3882
Кол-во негативных: 2130
Доля негативных к общему числу: 0.35429141716566864
--------------------------------------
'entities'


In [ ]:
train = gpt_train + ne5_train + nerel_train + \
 nerel_bio_train + person1000_train + rudrec_train

random.shuffle(train)

In [ ]:
train[110]

{'source': 'gpt',
 'messages': [{'role': 'user',
   'content': 'в результате атаки погибли по меньшей мере семь военнослужащих и ранены 20 человек.'},
  {'role': 'bot', 'content': 'Я ознакомился с текстом.'},
  {'role': 'user', 'content': 'Что описывает персона в тексте?'},
  {'role': 'bot', 'content': '[]'},
  {'role': 'user', 'content': 'Что описывает люди в тексте?'},
  {'role': 'bot', 'content': '["военнослужащих", "человек"]'},
  {'role': 'user', 'content': 'Что описывает место в тексте?'},
  {'role': 'bot', 'content': '[]'}]}

In [ ]:
train = train[:32]
gpt_val = gpt_val[:3]

In [ ]:
with open("train.jsonl", 'w') as f:
    for item in train:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

with open("val.jsonl", 'w') as f:
    for item in gpt_val:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

## Скачиваем и устанавливаем зависимости

In [ ]:
#@title Скачиваем и устанавливаем зависимости

from pathlib import Path

!git clone https://github.com/IlyaGusev/rulm.git
!git clone https://github.com/ggerganov/llama.cpp.git

self_instruct_dir = Path('rulm/self_instruct').resolve()

!cd {self_instruct_dir} && pip install -q -r ../requirements.txt
!pip install -q accelerate==0.27.2
!pip install -Uq transformers

Cloning into 'rulm'...
remote: Enumerating objects: 3338, done.
remote: Counting objects: 100% (808/808), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 3338 (delta 690), reused 713 (delta 626), pack-reused 2530
Receiving objects: 100% (3338/3338), 1.23 MiB | 3.99 MiB/s, done.
Resolving deltas: 100% (2221/2221), done.
Filtering content: 100% (79/79), 201.90 MiB | 65.12 MiB/s, done.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 23879, done.
remote: Counting objects: 100% (8388/8388), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 23879 (delta 8188), reused 8102 (delta 8046), pack-reused 15491
Receiving objects: 100% (23879/23879), 42.37 MiB | 25.73 MiB/s, done.
Resolving deltas: 100% (16957/16957), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.

## Скачиваем базовую модель LLaMA2 7B и чиним её кон

In [ ]:
from huggingface_hub import login
login()

In [ ]:
content_dir = Path('.').resolve()

from huggingface_hub import snapshot_download

model_dir = content_dir / "mistral"
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
snapshot_download(repo_id=base_model, local_dir=model_dir, ignore_patterns=["LICENSE", "README.md", ".gitattributes"])

patch_model_config = True

if patch_model_config:
    replacements = {
        "tokenizer_config.json": {
            "tokenizer_class": "LlamaTokenizer",
            "model_max_length": 3072,
            "padding_side": "left",
            "bos_token": "<s>",
            "eos_token": "</s>",
            "unk_token": "<unk>",
            "clean_up_tokenization_spaces": False,
            "special_tokens_map_file": "special_tokens_map.json",
        },
        "special_tokens_map.json": {
            "bos_token": "<s>",
            "eos_token": "</s>",
            "pad_token": "<unk>",
            "sep_token": "<s>",
            "unk_token": "<unk>",
        }
    }

    print('Patching model config...')
    for filename, new_content in replacements.items():
        print(f'{filename}:')
        with (model_dir / filename).open() as fp:
            old_content = json.load(fp)
            print(f'    Original content: {old_content}')
            if old_content == new_content:
                print('    Already patched, skipping')
        print(f'    Updated content:  {new_content}')
        with (model_dir / filename).open('w') as fp:
            json.dump(new_content, fp, indent=4)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Patching model config...
tokenizer_config.json:
    Original content: {'add_bos_token': True, 'add_eos_token': False, 'added_tokens_decoder': {'0': {'content': '<unk>', 'lstrip': False, 'normalized': False, 'rstrip': False, 'single_word': False, 'special': True}, '1': {'content': '<s>', 'lstrip': False, 'normalized': False, 'rstrip': False, 'single_word': False, 'special': True}, '2': {'content': '</s>', 'lstrip': False, 'normalized': False, 'rstrip': False, 'single_word': False, 'special': True}}, 'additional_special_tokens': [], 'bos_token': '<s>', 'clean_up_tokenization_spaces': False, 'eos_token': '</s>', 'legacy': True, 'model_max_length': 1000000000000000019884624838656, 'pad_token': None, 'sp_model_kwargs': {}, 'spaces_between_special_tokens': False, 'tokenizer_class': 'LlamaTokenizer', 'unk_token': '<unk>', 'use_default_system_prompt': False, 'chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exce

## Уменьшаем размер батча и лимит токенов, чтобы поместиться в Colab, и длительность обучения для демки

In [ ]:
original_config_path = self_instruct_dir / 'configs/saiga2_7b.json'

with original_config_path.open('r') as fp:
    config = json.load(fp)

# Colab adjustments
config['trainer']['per_device_train_batch_size'] = 1
config['trainer']['per_device_eval_batch_size'] = 1
config['trainer']['gradient_accumulation_steps'] = 32
config["trainer"]["eval_accumulation_steps"] = 32
config["trainer"]["learning_rate"] = 0.000025
config["trainer"]["warmup_ratio"] = 0.04
config["trainer"]["dataloader_drop_last"] = True
config["trainer"]["max_grad_norm"] = 2
config["trainer"]["fp16"] = True
config["trainer"]["bf16"] = False
config["trainer"]["weight_decay"] = 0.005
del config["trainer"]["warmup_steps"]

del config["trainer"]["save_steps"]
del config["trainer"]["eval_steps"]


config["trainer"]["evaluation_strategy"] = "epoch"
config["trainer"]["logging_strategy"] = "steps"
config["trainer"]["logging_first_step"] = True
config["trainer"]["save_strategy"] = "epoch"


config['max_tokens_count'] = 3072
config['model_name'] = str(model_dir)

# Demo adjustments
# del config['trainer']['eval_steps']
config['trainer']['logging_steps'] = 10
config['trainer']['num_train_epochs'] = 1

config["templates_path"] = "internal_prompts/saiga_v2.json"

config["lora"] = {
        "r": 8,
        "lora_alpha": 16,
        "lora_dropout": 0.05,
        "bias": "none",
        "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"],
        "task_type": "CAUSAL_LM"
    }

config["model_name"] = "mistralai/Mistral-7B-Instruct-v0.2"

config_path = self_instruct_dir / 'configs/mistral_hands.json'

with config_path.open('w') as fp:
    json.dump(config, fp, indent=4)

In [ ]:
config

{'trainer': {'evaluation_strategy': 'epoch',
  'per_device_train_batch_size': 1,
  'per_device_eval_batch_size': 1,
  'gradient_accumulation_steps': 32,
  'logging_steps': 10,
  'learning_rate': 2.5e-05,
  'num_train_epochs': 1,
  'lr_scheduler_type': 'cosine',
  'fp16': True,
  'bf16': False,
  'torch_compile': False,
  'optim': 'adamw_torch',
  'eval_accumulation_steps': 32,
  'warmup_ratio': 0.04,
  'dataloader_drop_last': True,
  'max_grad_norm': 2,
  'weight_decay': 0.005,
  'logging_strategy': 'steps',
  'logging_first_step': True,
  'save_strategy': 'epoch'},
 'lora': {'r': 8,
  'lora_alpha': 16,
  'lora_dropout': 0.05,
  'bias': 'none',
  'target_modules': ['q_proj', 'v_proj', 'k_proj', 'o_proj'],
  'task_type': 'CAUSAL_LM'},
 'load_in_8bit': True,
 'only_target_loss': True,
 'mode': 'chat',
 'templates_path': 'internal_prompts/saiga_v2.json',
 'model_name': 'mistralai/Mistral-7B-Instruct-v0.2',
 'model_type': 'causal',
 'max_tokens_count': 3072}

In [ ]:
original_template_path = self_instruct_dir / "internal_prompts/saiga_v2.json"

with original_template_path.open("r") as fp:
    config = json.load(fp)

config["system_prompt"] = "Ты полезная система извлечения информации из текста."

with original_template_path.open("w") as fp:
    json.dump(config, fp, indent=4)

# Запускаем обучение!

In [ ]:
#@title Запускаем обучение!

output_dir = Path("/content/drive/MyDrive/diploma_finetunes/mistral_hands_has_neg")

!cd {self_instruct_dir} && python3 -m src.train \
    --config-file {config_path.relative_to(self_instruct_dir)} \
    --train-file {content_dir / 'train.jsonl'} \
    --val-file {content_dir / 'val.jsonl'} \
    --output-dir {output_dir}

assert (output_dir / 'adapter_config.json').exists()

2024-05-06 14:05:04.079769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 14:05:04.079823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 14:05:04.081471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 14:05:05.221255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you wan

# Исправляем конфиг инференса обученной модели

In [ ]:
with (output_dir / 'generation_config.json').open('w') as fp:
    json.dump({
        "pad_token_id": 0,
        "bos_token_id": 1,
        "eos_token_id": 2,
        "temperature": 0.2,
        "top_p": 0.9,
        "top_k": 40,
        "do_sample": True,
        "max_new_tokens": 1536,
        "repetition_penalty": 1.1,
        "no_repeat_ngram_size": 15,
    }, fp, indent=4)

In [ ]:
#@title Склеиваем вместе обученные адаптеры с базовой моделью, сохраняем результат в формат PyTorch

merged_model_name = 'merged.pt'

!cd {self_instruct_dir} && python -m src.tools.merge_lora \
    {output_dir} \
    {output_dir / merged_model_name} \
#     --device=cuda \
#     --enable_offloading

assert (output_dir / merged_model_name).exists()